In [1]:
#!pip install celery

In [2]:
#!sudo rabbitmqctl add_user barnabas barnabaspwd
#!sudo rabbitmqctl set_permissions -p / barnabas "." "." ".*"

In [3]:
!sudo rabbitmqctl list_users

Listing users ...
barnabas	[]
guest	[administrator]
myguest	[]
myguest2	[]


on worker machine 1 start: <br>
celery -A test_workers worker --loglevel=info --concurrency=3

on worker machine 2 start: <br>
celery -A test_workers worker --loglevel=info --concurrency=4


In [4]:
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import json

import celery
from test_workers import message_to_workers1, message_to_workers2

import math
import socket
import os


In [5]:
celery.current_app.control.inspect().ping()

{'celery@ip-172-31-3-73': {'ok': 'pong'},
 'celery@ip-172-31-0-150': {'ok': 'pong'}}

In [6]:
mykeys=list(celery.current_app.control.inspect().stats().keys())
print(mykeys)

['celery@ip-172-31-0-150', 'celery@ip-172-31-3-73']


In [7]:
socket.gethostbyname(socket.gethostname())

'172.31.0.150'

In [8]:
mydict=celery.current_app.control.inspect().stats()[mykeys[0]]
mydict['pool']

{'max-concurrency': 3,
 'processes': [8176, 8177, 8178],
 'max-tasks-per-child': 'N/A',
 'put-guarded-by-semaphore': False,
 'timeouts': [0, 0],
 'writes': {'total': 0,
  'avg': '0.00%',
  'all': '',
  'raw': '',
  'strategy': 'fair',
  'inqueues': {'total': 3, 'active': 0}}}

In [9]:
mydict=celery.current_app.control.inspect().stats()[mykeys[1]]
mydict['pool']

{'max-concurrency': 4,
 'processes': [5218, 5219, 5221, 5223],
 'max-tasks-per-child': 'N/A',
 'put-guarded-by-semaphore': False,
 'timeouts': [0, 0],
 'writes': {'total': 0,
  'avg': '0.00%',
  'all': '',
  'raw': '',
  'strategy': 'fair',
  'inqueues': {'total': 4, 'active': 0}}}

In [10]:
#process id
os.getpid()

8362

In [ ]:
for iter in range(0,30):
    myresult1=message_to_workers1.delay(str(1000+iter)) 
    result1=myresult1.get()
    
    myresult2=message_to_workers2.delay(str(1000+iter))
    result2=myresult2.get()
    print('***')
    print(result1)
    print(result2)
    print('***')

***
['172.31.3.73_ForkPoolWorker-2_**5219**_1000_730_72646_****']
['172.31.0.150_ForkPoolWorker-2_**8177**_1000_136_72679_****']
***
***
['172.31.3.73_ForkPoolWorker-1_**5218**_1001_730_72699_****']
['172.31.0.150_ForkPoolWorker-3_**8178**_1001_136_72731_****']
***
***
['172.31.3.73_ForkPoolWorker-2_**5219**_1002_246_72748_****', '172.31.3.73_ForkPoolWorker-2_**5219**_1000_730_72646_****']
['172.31.0.150_ForkPoolWorker-1_**8176**_1002_136_72778_****']
***
***
['172.31.3.73_ForkPoolWorker-1_**5218**_1003_246_72796_****', '172.31.3.73_ForkPoolWorker-1_**5218**_1001_730_72699_****']
['172.31.0.150_ForkPoolWorker-2_**8177**_1003_488_72826_****', '172.31.0.150_ForkPoolWorker-2_**8177**_1000_136_72679_****']
***
